# Pistes de réflexion explorées

Dans cette première partie nous allons décrire plusieurs pistes qui ont été explorées dans le cadre de ce projet de Machine Learning. Certaines de ces pistes ont abouti à des résultats satisfaisants et ont servi à construire peu à peu les lignes de code présentes plus loin dans ce notebook. D'autres n'ont pas semblé assez intéressantes et n'ont pas été explorées plus en détails.

## Création de modèles spécifiques

### Modèles spécifiques par secteurs

Après avoir observé et lancé le notebook du benchmark, nous avons assez vite pensé à créer des modèles spécifiques pour des données se ressemblant entre elles. Notre première intuition a été de créer 12 modèles pour les 12 secteurs, en imaginant que les actions d'un même secteur ont un comportement similaire. Une soumission à 51,73% sur la base de cette idée nous a conforté dans l'idée de la creuser.

### Modèles spécifiques par catégories, selon la quantité de données d'entraînement

Par _catégories_ on entend ici les groupes dans lesquels les données sont classées : _Secteur_, _Groupe industriel_, etc. Cette nouvelle idée consiste en une amélioration de la précédente. Puisque des modèles spécifiques par secteurs ont donné de bons résultats, pourquoi ne pas descendre d'un cran et créer des modèles par groupes industriels ? Et pourquoi pas descendre encore aux industries ? Une nouvelle question est vite apparue : quand s'arrêter ?

Pour adresser des réponses à ces questions, nous avons transformé le code précédent en y introduisant des constantes de _seuil_. Un seuil action par exemple détermine à partir de combien de données disponibles dans le set d'entaînement pour une action donnée un modèle spécifique sera créé pour cette action. Idem pour les autres catégories, en créant des modèles pour les catégories les plus petites possible.

_Exemple : Si on remonte l'arborescence de l'action 2884 **(156 données)**, elle appartient à la sous-industrie 107 **(758 données)**, à l'industrie 46 **(30325 données)**, au groupe indutriel 16 **(39290 données)** et au secteur 7 **(87903 données)**. Imaginons qu'on fixe un même seuil pour toutes les catégories égal à 36000. On remonte les catégories depuis l'action jusqu'à dépasser ce seuil. Ainsi sur cet exemple, les données de l'action 2884 seront prédites avec un modèle spécifique au groupe industriel 16._

Sur cette base, nous avons fait varier 2 seuils en validation croisée afin de trouver leur combinaison optimale : un seuil_secteur et un seuil_autre, commun à toutes les autres catégories. Voici un extrait des résultats de validation croisée obtenus :
![Recherche double seuils](Recherche_double_seuil_affine_seuils_hauts.PNG)
Remarque : les précisions obtenues sont relativement faibles comparées aux autres validations croisées effectuées. Ceci est dû à un découpage particulier pour créer les folds, sur lequel nous reviendrons un peu plus tard.

Le cas indiqué par la flèche correspond à 0 modèle secteur et 1 modèle groupe industriel. Dans ce cas les données du groupe industriel 16 sont prédites avec un modèle spécifique, et toutes les autres données avec un modèl général, entraîné sur le dataset entier.

Malheureusement la soumission avec ces seuils n'a pas dépassé 51,10%.

La solution retenue a été un essai arbitraire avec seuil_secteur=0 (chaque secteur a un modèle spécifique associé) et seuil_autre=36000 (seul le groupe_indsutriel 16 a un modèle spécifique). Avec ces paramètres, le score de notre soumission a été de 51,82%.

Ci-dessous la recherche de seuil_autre pour seuil_secteur fixé à 0 qui nous a fait retenir la valeur 36000 (il faut multiplier les valeurs en abscisse par 1,2 pour passer du KFold aux seuils réels) :
![Recherche seuil_autre](Accuracy_per_seuil_5Folds_36000.PNG)

### Clustering

Toujours dans l'idée de rassembler des données qui se ressemblent pour créer des modèles spécifiques, nous avons pensé à faire du clustering sur les datasets. Notre intutition était donc de créer des modèles spécifiques en regroupant les données non pas arbitrairment mais par ressemblance. Concrètement : créer des groupes de données par clusering plutôt que de se contenter de les regrouper par secteurs.

Après avoir essayé différents algorithmes de clustering nous sommes restés sur du KMeans, qui présentait des résultats satisfaisants et un temps de calcul décent. Afin d'obtenir des résultats plus rapides encore pour les validations croisées et recherches de paramètres opti, nous avons utilisé le MiniBatchKMeans. Cet algorithme est une version accélérée du KMeans, qui se fait au dépends d'un peu de précision.

Voici une partie des résultats obtenus en faisant varier seuil_autre et le nombre k de clusters créés, pour seuil_secteur fixé à 0 :
![Recherche k KMeans](Accuracy_per_k_seuil_kmeans_4Folds.PNG)

La soumission de cette solution n'a pas dépassé 50,98%. Nous avons donc abandonné la piste du clustering.

## Autres classificateurs

Nous avons essayé assez tôt d'appliquer différents classificateurs aux données, avec leurs paramètres par défauts, afin de les comparer rapidement et de sélectionner les plus adaptés à notre problème.
![Essais classificateurs](Accuracy_per_classifier_5Folds.PNG)
Sur la base de ces essais, nous avons continué d'explorer la random forest (qui a donné tous les résultats de la partie précédente), et de regarder également du côté du KNN et de l'adaboost.

### K Nearest Neighbors

De la même manière que nous avons fait varier divers paramètres jusque là, nous avons réalisé des validations croisées pour déterminer le k optimal. Après divers essais nous avons retenu deux valeurs de k qui ont mené à des soumissions : k=9 et k=15. Ces deux soumissions n'ont pas dépassé 50,35%. Ce sont aujourd'hui encore nos pires résultats obtenus, ce qui a poussé le KNN à rejoindre la liste des pistes abandonnées.

### AdaBoost

Après avoir testé plusieurs algorithmes, l'algo AdaBoost semblait prometteur. Nous avons exploré cet algortihme avec plusieurs configurations tout en modifiant les paramètres. 
Cependant il est apparu qu'il ne convenait pas à l'exercice. Nous l'avons vite abandonné avec un record d'accuracy à 51,3. 

### Réseau neuronal avec Keras

Nous avons utilisé la bibliothèque Kéras afin d'entrainer un modèle général sur l'ensemble du train set. Après documentation, des réseaux neuronaux sont parfois utilisés pour des problèmes plus ou moins similaires. 

Nous avons testé plusieurs combinaisons nombre de couches/nombre de neuronnes par couche/fonction d'activation. Toujours avec un modèle séquenciel. L'accuracy sur le train set (qui est une majoration de l'accuracy sur le test set) n'a jamais dépassé 50,3% ce qui nous a conduit à abandonner la piste d'une classification via deep learning.

### VariationRandom Forest : ExtraTree

La documentation de sklearn recommande l'algorithme ExtraTree - similaire à RandomForest - pour éviter l'overfitting. Les premières tentatives plutôt prometteuses se sont rapprochées des résulats d'accuracy des modèles utilisant RandomForest pour des paramètres similaires. 

Dans ce genre d'exercice la limite entre un bon modèle et un modèle overfitté est très fine et l'enjeux et de se rapprocher de cette frontière sans la franchir. Dans le cas d'ExtraTree, certe l'overfitting n'a jamais été atteint mais il n'a pas non plus été approché, ce qui n'a pas permis de créer d'excellent modèle. La meilleur tentative avec ExtraTree s'est soldé par une accuracy plutôt honorable de 51,5%. Par la suite nous avons abandonné cet algorithme.

## Remarque importante sur les datasets

Lors de notre exploration des datasets fournis, nous avons fait quelques découvertes dont nous avons essayé de tenir compte lors de la création de nos modèles.

Le plus important d'après nous est le découpage entre les sets train et test. Ce découpage n'a manifestement pas été fait de manière aléatoire ou homogène. En effet, aucune **Date** n'apparaissant dans le train n'apparaît dans le test et inversement. Nous avons donc pensé que cela pouvait expliquer les scores de précision en sortie de validation croisée très supérieurs aux scores réels de soumission (~56% contre ~51%). Puisque nous faisions des validations croisées sur des folds créés aléatoirement, on retrouvait des dates identiques dans le train et le test set, créant ainsi une sorte d'overfitting comparé aux conditions réelles de soumission. C'est pourquoi un certain nombre de validations croisées ont été réalisées par la suite avec des folds créés non pas totalement aléatoirement, mais selon les dates.

Concrètement, nous avons utilisé la fonction KFold sur la liste des dates, puis récupéré les données selon leur date pour les associer au train ou au test set. En procédant ainsi, nos validations croisées donnaient des résultats bien plus proches des soumissions, autour de 51%. Nous avions ainsi plus confiance en nos résultats et avons lancé un certain nombre de recherches de paramètres optimaux dans ces conditions.

Il s'est finalement avéré que les paramètres (seuil, k, ...) déterminés de cette manière donnaient des résultats décevants une fois soumis sur le site du challenge. Nous nous sommes donc rabattus sur des validations croisées aléatoires, comme au début, en comparant des valeurs que nous savions trop élevées. Nous cherchons encore aujourd'hui à expliquer ces observations.

## Quelques détails avant de rentrer dans le vif du sujet

Avant de vous présenter notre code final, voici quelques autres détails qui ont été pensés, explorés puis testés pour aboutir à la version finale :

- On utilise la totalité des RET_t et VOL_t, soit 40 features, pour l'entraînement des modèles.


- Le nettoyage des données se fait simplement en enlevant les lignes avec au moins un RET_t ou un VOL_t supérieur à 1000. Cela ne supprime qu'une donnée du train set. Nous avons gardé ce seuil depuis le début sans le remettre en question, cela représente sûrmeent une petite piste d'amélioration.


- Les paramètres de la random forest ont été remis en question, notamment max_depth et max_features. Nous n'avons pa trouvé mieux que ceux proposés par le benchmark.


- Les new_features sont similaires à celles du benchmark, on n'a pas trouvé mieux qu'en considérer 5. La gestion de modèles spécifiques se fait de la manière suivante : 
    - Les new_features d'un modèle spécifique à une catégorie sont les médianes par date pour cette catégorie.
    - Les données utilisées pour le calcul des new_features d'une catégorie ne sont pas utilisées pour les catégories supérieures (ex: s'il existe un modèle groupe_ind 16, les new_features du secteur 7 sont calculées d'après les données du secteur 7 exceptées celles du groupe_ind 16).
    
    
- Un modèle spécifique est entraîné d'après toutes les données de la catégorie associée (ex: s'il existe un modèle groupe_ind 16, le modèle du secteur 7 est entraîné d'après les données du secteur 7 y compris celles du groupe_ind 16).


- Les valeurs prédites ne sont pas centrées sur la médiane par dates comme le benchmark. AU lieu de ça on attribue directement la valeur True/False la plus probable d'après le modèle utilisé.

# Obtention des résultats finaux

Les lignes de code suivantes permettent de recréer notre soumission à 51,82%. C'est la solution que nous avons retenue et qui a fourni notre 3e score le plus élevé.

## Import des librairies

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import collections
from sklearn.metrics import accuracy_score
import numpy as np

## Chargement des données

In [2]:
x_train = pd.read_csv('../x_train.csv', index_col='ID')
y_train = pd.read_csv('../y_train.csv', index_col='ID')
train = pd.concat([x_train, y_train], axis=1)
test = pd.read_csv('../x_test.csv', index_col='ID')
train.head()

,DATE,STOCK,INDUSTRY,INDUSTRY_GROUP,SECTOR,SUB_INDUSTRY,RET_1,VOLUME_1,RET_2,VOLUME_2,...,VOLUME_16,RET_17,VOLUME_17,RET_18,VOLUME_18,RET_19,VOLUME_19,RET_20,VOLUME_20,RET
ID,,,,,,,,,,,,,,,,,,,,,
0,0,2,18,5,3,44,-0.015748,0.147931,-0.015504,0.179183,...,0.630899,0.003254,-0.379412,0.008752,-0.110597,-0.012959,0.174521,-0.002155,-0.000937,True
1,0,3,43,15,6,104,0.003984,NaN,-0.090580,NaN,...,NaN,0.003774,NaN,-0.018518,NaN,-0.028777,NaN,-0.034722,NaN,True
2,0,4,57,20,8,142,0.000440,-0.096282,-0.058896,0.084771,...,-0.010336,-0.017612,-0.354333,-0.006562,-0.519391,-0.012101,-0.356157,-0.006867,-0.308868,False
3,0,8,1,1,1,2,0.031298,-0.429540,0.007756,-0.089919,...,0.012105,0.033824,-0.290178,-0.001468,-0.663834,-0.013520,-0.562126,-0.036745,-0.631458,False
4,0,14,36,12,5,92,0.027273,-0.847155,-0.039302,-0.943033,...,-0.277083,-0.012659,0.139086,0.004237,-0.017547,0.004256,0.579510,-0.040817,0.802806,False


In [3]:
ret=[f'RET_{k}' for k in range(20,0,-1)]
vol=[f'VOLUME_{k}' for k in range(20,0,-1)]
target = 'RET'
categories = ['STOCK','SUB_INDUSTRY','INDUSTRY','INDUSTRY_GROUP','SECTOR']
features_base = ret+vol

## Paramètres

In [4]:
nb_shifts = 5 #nombre de features supplémentaires
seuil_ret = 1000
seuil_vol = 1000
rf_params = {
'n_estimators': 500,
'max_depth': 2**3,
'random_state': 0,
'n_jobs': -1
}
seuil_sector = 0
seuil_other = 36000

In [5]:
seuil = [seuil_other,seuil_other,seuil_other,seuil_other,seuil_sector]

## Nettoyage des données

In [6]:
train_cl = train.copy()
test_cl = test.copy()

for ind_groupe in np.intersect1d(train['INDUSTRY_GROUP'].unique(),test['INDUSTRY_GROUP'].unique()):
    for ret_t,vol_t in zip(ret,vol):
            med_ret = train_cl[ret_t][train['INDUSTRY_GROUP']==ind_groupe].median()
            med_vol = train_cl[vol_t][train['INDUSTRY_GROUP']==ind_groupe].median()

            train_cl.loc[:,ret_t] = train_cl.loc[:,ret_t].fillna(med_ret)
            train_cl.loc[:,vol_t] = train_cl.loc[:,vol_t].fillna(med_vol)
            test_cl.loc[:,ret_t] = test_cl.loc[:,ret_t].fillna(med_ret)
            test_cl.loc[:,vol_t] = test_cl.loc[:,vol_t].fillna(med_vol)

train_cl  = train_cl[np.all([train_cl[col]<seuil_ret for col in ret],axis=0)]
train_cl  = train_cl[np.all([train_cl[col]<seuil_vol for col in vol],axis=0)]

scaler = StandardScaler()
train_cl.loc[:,ret+vol] = scaler.fit_transform(train_cl.loc[:,ret+vol])
test_cl.loc[:,ret+vol] = scaler.fit_transform(test_cl.loc[:,ret+vol])

## Sélection des groupes à conserver selon les seuils

In [7]:
model_labels = [[] for ind_cat in range(len(categories))]

for ind_cat,cat in enumerate(categories):
    
    count_group = pd.Series(collections.Counter(train_cl[cat])).sort_index()
    for group in count_group.index:
        
        if count_group.loc[group]>=seuil[ind_cat]:
            model_labels[ind_cat].append(group)
            
    print(f"{len(model_labels[ind_cat])} modèles spécifiques de {cat} vont être entraînés")

0 modèles spécifiques de STOCK vont être entraînés
0 modèles spécifiques de SUB_INDUSTRY vont être entraînés
0 modèles spécifiques de INDUSTRY vont être entraînés
1 modèles spécifiques de INDUSTRY_GROUP vont être entraînés
12 modèles spécifiques de SECTOR vont être entraînés


## Création des new features par groupes

### Création de dataframes (vides) pour les new features

In [8]:
new_feat_train = pd.DataFrame()
new_feat_test = pd.DataFrame()
shifts = range(1,nb_shifts+1)
new_features = []
statistics = ['median']
target_feature = 'RET'

for shift in shifts:
    for stat in statistics:
        name = f'{target_feature}_{shift}_CATEGORIE_{stat}'
        new_features.append(name)
        for data in [new_feat_train,new_feat_test]:
            data[name] = None

### Calcul des new features pour chaque groupe

In [9]:
train_temp = train_cl.copy()
test_temp = test_cl.copy()
#Copie des datasets d'où on va peu à peu retirer les données après leur utilisation

for ind_cat,cat in enumerate(categories):
    for ind_group,group in enumerate(model_labels[ind_cat]):
        
        x_train_cl = train_temp.copy()[train_temp[cat]==group]
        y_train_cl = train_temp[target][train_temp[cat]==group]
        
        x_test = test_temp.copy().loc[test_temp[cat]==group,:]

        shifts = range(1,nb_shifts+1)
        statistics = ['median']
        gb_features = [categories[ind_cat],'DATE']
        target_feature = 'RET'
        
        for shift in shifts:
            for stat in statistics:
                name = f'{target_feature}_{shift}_CATEGORIE_{stat}'
                feat = f'{target_feature}_{shift}'
                for data in [x_train_cl,x_test]:
                    if len(data)>0:
                        data[name] = data.groupby(gb_features)[feat].transform(stat)
                
        new_feat_train = pd.concat([new_feat_train,x_train_cl[new_features]])
        new_feat_test = pd.concat([new_feat_test,x_test[new_features]])
        
        train_temp = train_temp[train_temp[cat]!=group]
        test_temp = test_temp[test_temp[cat]!=group] #On supprime les données utilisées

### Calcul des new features pour le modèle général si nécessaire

In [10]:
for data_temp,new_feat_data in zip([train_temp,test_temp],[new_feat_train,new_feat_test]):

    if len(data_temp)>0:

        x_data = data_temp.copy()

        shifts = range(1,nb_shifts+1)
        statistics = ['median']
        gb_features = ['SECTOR','DATE']
        target_feature = 'RET'

        for shift in shifts:
            for stat in statistics:
                name = f'{target_feature}_{shift}_CATEGORIE_{stat}'
                feat = f'{target_feature}_{shift}'
                x_data[name] = x_data.groupby(gb_features)[feat].transform(stat)

        new_feat_data = pd.concat([new_feat_data,x_data[new_features]])

### Ajout des new features aux datasets

In [11]:
new_feat_train = new_feat_train.loc[train_cl.index]
new_feat_test = new_feat_test.loc[test_cl.index] #Réindexage pour associer les new features aux bonnes lignes

train_cl[new_features] = new_feat_train
test_cl[new_features] = new_feat_test #Ajout des colonnes new features au dataset d'entraînement

## Entraînement des modèles

### Modèles par groupes

In [14]:
models = [{} for ind_cat in range(len(categories)+1)]
features = features_base + new_features

for ind_cat,cat in enumerate(categories):
    for ind_group,group in enumerate(model_labels[ind_cat]):
        
        x_train_cl = train_cl.copy()[train_cl[cat]==group]
        y_train_cl = train_cl[target][train_cl[cat]==group]

        model = RandomForestClassifier(**rf_params)
        model.fit(x_train_cl[features], y_train_cl)
        models[ind_cat][group] = model

        print(f"Modèle {cat} {group} entraîné : {len(x_train_cl)} données")

Modèle INDUSTRY_GROUP 16 entraîné : 39290 données
Modèle SECTOR 0 entraîné : 6304 données
Modèle SECTOR 1 entraîné : 21264 données
Modèle SECTOR 2 entraîné : 18967 données
Modèle SECTOR 3 entraîné : 55473 données
Modèle SECTOR 4 entraîné : 63519 données
Modèle SECTOR 5 entraîné : 17295 données
Modèle SECTOR 6 entraîné : 55122 données
Modèle SECTOR 7 entraîné : 87903 données
Modèle SECTOR 8 entraîné : 70843 données
Modèle SECTOR 9 entraîné : 5555 données
Modèle SECTOR 10 entraîné : 13295 données
Modèle SECTOR 11 entraîné : 3054 données


### Modèle général si nécessaire

In [ ]:
if len(train_temp)>0:
    
    x_train_cl = train_cl.copy()
    y_train_cl = train_cl[target]

    model = RandomForestClassifier(**rf_params)
    model.fit(x_train_cl[features], y_train_cl)
    models[5]['general'] = model
    print(f"Modèle général entraîné : {len(x_train_cl)} données")

## Prédictions

### Prédictions par groupe

In [15]:
test_temp = test_cl.copy()
y_pred = pd.Series()

for ind_cat,cat in enumerate(categories):
    for ind_group,group in enumerate(model_labels[ind_cat]):
        
        x_test = test_temp.copy().loc[test_temp[cat]==group,:]
        if len(x_test)>0:
            
            index = x_test.index
            model = models[ind_cat][group]
            y_pred = pd.concat([y_pred,pd.Series(model.predict(x_test[features]),index=x_test.index)])
            print(f"{len(x_test)} données prédites avec le modèle {cat} {group}")

            test_temp = test_temp[test_temp[cat]!=group] #On supprime les données utilisées

17939 données prédites avec le modèle INDUSTRY_GROUP 16
2177 données prédites avec le modèle SECTOR 0
11844 données prédites avec le modèle SECTOR 1
9176 données prédites avec le modèle SECTOR 2
27312 données prédites avec le modèle SECTOR 3
27951 données prédites avec le modèle SECTOR 4
7977 données prédites avec le modèle SECTOR 5
27706 données prédites avec le modèle SECTOR 6
23248 données prédites avec le modèle SECTOR 7
30092 données prédites avec le modèle SECTOR 8
3358 données prédites avec le modèle SECTOR 9
5733 données prédites avec le modèle SECTOR 10
3916 données prédites avec le modèle SECTOR 11


### Prédictions avec le modèle général si nécessaire

In [16]:
if len(test_temp)>0:
    x_test = test_temp.copy()
    index = x_test.index
    model = models[5]['general']
    
    y_pred = pd.concat([y_pred,pd.Series(model.predict(x_test[features]),index=x_test.index)])
    print(f"{len(x_test)} données prédites avec le modèle général")

## Préparation des données prédites pour soumission

In [17]:
y_pred = y_pred[test_cl.index]
y_pred.name = target
# y_pred.to_csv('./nom_submit.csv', index=True, header=True)